In [16]:
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re
import hashlib
from pymongo import MongoClient

In [17]:
client = MongoClient()
dbchessResultsSchool = client['db_chessResultsSchoolDB']
collectionChessResultsPass = dbchessResultsSchool['collection_chessResultsSchoolPass']
collectionChessResultsFail = dbchessResultsSchool['collection_chessResultsSchoolFail']

In [6]:
driver = webdriver.Chrome()
driver.get("http://chess-results.com/TurnierSuche.aspx?lan=1")
sleep(10)
# countrySelect = driver.find_element_by_xpath('//select[@id="_ctl0_P1_combo_land"]')
# sleep(5)
# driver.refresh()
# countrySelect.click()
# sleep(5)
driver.find_element_by_xpath('//select[@id="_ctl0_P1_combo_land"]//option[starts-with(text(),"India")]').click()
driver.find_element_by_tag_name('body').send_keys(Keys.ESCAPE)
driver.execute_script("window.scrollBy(0,180);")

In [7]:
driver.find_element_by_xpath('//*[@id="_ctl0_P1_txt_bez"]').send_keys('cbse')

In [8]:
driver.find_element_by_xpath('//*[@id="_ctl0_P1_cb_suchen"]').click()

In [9]:
blue_links = []
all_blue_links = driver.find_elements_by_xpath("//div[@class='defaultDialog']//table//tr//td[1]//a")
for each in all_blue_links:
    blue_links.append(each.get_attribute('href'))

In [11]:
# total tournaments links for "cbse" keyword/tournament :  

for each_tournament_link in blue_links:
    print(each_tournament_link)

http://chess-results.com/tnr373002.aspx?lan=1
http://chess-results.com/tnr368081.aspx?lan=1
http://chess-results.com/tnr368080.aspx?lan=1
http://chess-results.com/tnr368079.aspx?lan=1
http://chess-results.com/tnr368078.aspx?lan=1
http://chess-results.com/tnr365736.aspx?lan=1
http://chess-results.com/tnr365735.aspx?lan=1
http://chess-results.com/tnr365734.aspx?lan=1
http://chess-results.com/tnr314172.aspx?lan=1
http://chess-results.com/tnr314173.aspx?lan=1
http://chess-results.com/tnr314176.aspx?lan=1
http://chess-results.com/tnr314177.aspx?lan=1
http://chess-results.com/tnr314174.aspx?lan=1
http://chess-results.com/tnr314175.aspx?lan=1
http://chess-results.com/tnr314214.aspx?lan=1
http://chess-results.com/tnr314213.aspx?lan=1
http://chess-results.com/tnr314501.aspx?lan=1
http://chess-results.com/tnr314492.aspx?lan=1
http://chess-results.com/tnr314500.aspx?lan=1
http://chess-results.com/tnr314491.aspx?lan=1
http://chess-results.com/tnr314496.aspx?lan=1
http://chess-results.com/tnr314503

In [12]:
print("total no of cbse blue links for tournament are ")
print(len(blue_links))

total no of cbse blue links for tournament are 
274


In [13]:
blue_links[0:6] # stored all the tournaments links in list named as 'blue_links'

['http://chess-results.com/tnr373002.aspx?lan=1',
 'http://chess-results.com/tnr368081.aspx?lan=1',
 'http://chess-results.com/tnr368080.aspx?lan=1',
 'http://chess-results.com/tnr368079.aspx?lan=1',
 'http://chess-results.com/tnr368078.aspx?lan=1',
 'http://chess-results.com/tnr365736.aspx?lan=1']

In [18]:
# starting from uneven code break : 

# driver = webdriver.Chrome()

In [19]:
for each_url_of_blue_tournament in blue_links:
    

    if (collectionChessResultsPass.find_one({'tournamentURL':str(each_url_of_blue_tournament)})):
        print("tournament already covered in dbchessResultsSchool")
        continue
    
    elif (collectionChessResultsFail.find_one({'tournamentURLFail':str(each_url_of_blue_tournament)})):
        continue
    
    else:        
       
        print("Moving the driver to clicking blue link of tournament")
        driver.get(each_url_of_blue_tournament) # clicking the blue link in order
        sleep(5)

        try:
            print("Clicking show tournament details button if exists")
            driver.find_element_by_xpath('//input[@value="Show tournament details"]').click()  # clicked show tournament details button
            print("Waiting for 5 seconds")
            sleep(5)

        except:
            print("does not exists")
            pass


        if (len(re.findall(r"Team", str(driver.page_source))) > 0):
            print("Evidence for schools' presence exists in links in a table on screen ")
            all_schools_blue_link_list = []  # schoolname, linkForThatSchool

            # we are onto a table of schools where schools became links in blue now
            try:
                for each_school_url in driver.find_elements_by_xpath("//table[@class='CRs1'][1]//tr//a"):   # getting all school links in blue and storing them into a list
                    temp = {}
                    temp['school name'] = each_school_url.text   
                    temp['link for that school'] = each_school_url.get_attribute('href')
                    all_schools_blue_link_list.append(temp)

            except:
                print("error in finding the school blue links")


                dict_for_school_wise_failure = {}
                mongoIdHash = hashlib.md5(each_url_of_blue_tournament.encode('utf-8')).hexdigest()

                dict_for_school_wise_failure['_id'] = mongoIdHash
                dict_for_school_wise_failure['tournamentURLFail'] = str(each_url_of_blue_tournament)
                dict_for_school_wise_failure['failSchoolURL'] = None
                collectionChessResultsFail.insert_one(dict_for_school_wise_failure)

                continue




            print("details for all those schools for our tournament", all_schools_blue_link_list)



            for i in range(len(all_schools_blue_link_list)): 
                try:

                    driver.get(all_schools_blue_link_list[i]['link for that school'])    # clicking the school in blue link and moving to its url
                    sleep(5)


                    print("\n",'school name is ', all_schools_blue_link_list[i]['school name'])
                    print("url for this school is ",all_schools_blue_link_list[i]['link for that school'] )

                    dict_for_school_wise_pass = {}

                    mongoIdHash = hashlib.md5(str(all_schools_blue_link_list[i]['link for that school'] + all_schools_blue_link_list[i]['school name'] ).encode('utf-8')).hexdigest()
                    dict_for_school_wise_pass['_id'] = mongoIdHash
                    dict_for_school_wise_pass['schoolName'] = all_schools_blue_link_list[i]['school name']
                    dict_for_school_wise_pass['schoolURL'] = all_schools_blue_link_list[i]['link for that school'] 
                    dict_for_school_wise_pass['tournamentURL'] = str(each_url_of_blue_tournament)                

                    data_all_combined = ''

                    print("getting tournament name")
                    tournament_name = driver.find_element_by_xpath("//div[@class='defaultDialog']//h2").text
                    print("tournament name :",tournament_name)

                    dict_for_school_wise_pass['tournament_name'] = str(tournament_name)


                    for final_data_tabular in driver.find_elements_by_xpath("//table[@class='CRs1'][1]//tr//td"):  # for final table data extraction
                        data_all_combined = data_all_combined + " " +  final_data_tabular.text

                    dict_for_school_wise_pass['schoolName'] = all_schools_blue_link_list[i]['school name']
                    dict_for_school_wise_pass['schoolURL'] = all_schools_blue_link_list[i]['link for that school'] 
                    dict_for_school_wise_pass['tournamentURL'] = str(each_url_of_blue_tournament)
                    dict_for_school_wise_pass['dataForSchool'] = str(data_all_combined)
                    dict_for_school_wise_pass['schoolURLlink'] = str(all_schools_blue_link_list[i]['link for that school'])

                    print("Inserting into Mongodb collection with success")
                    collectionChessResultsPass.insert_one(dict_for_school_wise_pass)    # inserting into mongodb collection
                    print(dict_for_school_wise_pass)
                    print()


                    sleep(5)

                except:

                    print("Failure for this tournament  ", each_url_of_blue_tournament)

                    dict_for_school_wise_failure = {}
                    mongoIdHash = hashlib.md5(str(all_schools_blue_link_list[i]['link for that school']).encode('utf-8')).hexdigest()

                    dict_for_school_wise_failure["_id"] = mongoIdHash
                    dict_for_school_wise_failure['failSchoolURL'] = all_schools_blue_link_list[i]['link for that school']
                    dict_for_school_wise_failure['tournamentURLFail'] = str(each_url_of_blue_tournament)

                    collectionChessResultsFail.insert_one(dict_for_school_wise_failure)

                    continue

        else:
            print("No Evidence for school links in table present on screen!!!")
            print()
            continue







    

Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
tournament already covered in dbchessResultsSchool
tournament alread

Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for sc

Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
Moving the driver to clicking blue link of tournament
Clicking show tournament details button if exists
Waiting for 5 seconds
No evidence for school links in table present on screen
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
tournament already covered in dbchessResultsSchool
tournament already c

Inserting into Mongodb collection with success
{'_id': '6f2919a039623819e012f6c0960175e8', 'schoolName': 'JAWAHAR VIDYALYA CHENNAI', 'schoolURL': 'http://chess-results.com/tnr27280.aspx?lan=1&art=20&snr=1', 'tournamentURL': 'http://chess-results.com/tnr27280.aspx?lan=1', 'tournament_name': 'CBSE NATIONAL U-19 MIXED CHESS TOURNAMENT-2009', 'dataForSchool': '   1. JAWAHAR VIDYALYA CHENNAI (RtgAvg:2132, TB1: 12 / TB2: 19,5) Bo.  Name Rtg FED 1 2 3 4 5 6 Pts. Games 1  AJAY SHREEKANTH 2138 IND 0 ½ 1 0 1 1 3,5 6 2  D.S. SHASHAANK 2123 IND 1 1 1 1 ½ 0 4,5 6 3  S. KAUSHIK 2119 IND 1 1 1 1 1 1 6,0 6 4  G.AAKASH 2148 IND 1 1 1 1 1 ½ 5,5 6', 'schoolURLlink': 'http://chess-results.com/tnr27280.aspx?lan=1&art=20&snr=1'}


 school name is  DPS SEC12 R.K.PURAM
url for this school is  http://chess-results.com/tnr27280.aspx?lan=1&art=20&snr=5
getting tournament name
tournament name : CBSE NATIONAL U-19 MIXED CHESS TOURNAMENT-2009
Inserting into Mongodb collection with success
{'_id': 'bcb384f3359d44e51

Inserting into Mongodb collection with success
{'_id': '23d0c122a46a6e5542b3a8227e1faeeb', 'schoolName': 'DELHI PUBLIC MATHURA N. DELHI', 'schoolURL': 'http://chess-results.com/tnr27280.aspx?lan=1&art=20&snr=7', 'tournamentURL': 'http://chess-results.com/tnr27280.aspx?lan=1', 'tournament_name': 'CBSE NATIONAL U-19 MIXED CHESS TOURNAMENT-2009', 'dataForSchool': '   10. DELHI PUBLIC MATHURA N. DELHI (RtgAvg:1605, TB1: 6 / TB2: 13,5) Bo.  Name Rtg FED 1 2 3 4 5 6 Pts. Games 1  J. GAUTAM 1500 IND 1 0 0 0 0 ½ 1,5 6 2  IQBAL 1656 IND 1 1 ½ 0 0 1 3,5 6 3  MEHAR JOT 1739 IND ½ 1 0 ½ 1 1 4,0 6 4  AGAMJOT 1525 IND 1 1 0 1 ½ 1 4,5 6', 'schoolURLlink': 'http://chess-results.com/tnr27280.aspx?lan=1&art=20&snr=7'}


 school name is  DALIMSS SUNBEAM VARANASI
url for this school is  http://chess-results.com/tnr27280.aspx?lan=1&art=20&snr=17
getting tournament name
tournament name : CBSE NATIONAL U-19 MIXED CHESS TOURNAMENT-2009
Inserting into Mongodb collection with success
{'_id': 'ff2752f6fc9b8d441d

...


 school name is  C.J.D.A.V.SCHOOL(MEERUT)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=2
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
{'_id': 'a750ddcbbee1d770b238d9be595bfad2', 'schoolName': 'C.J.D.A.V.SCHOOL(MEERUT)', 'schoolURL': 'http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=2', 'tournamentURL': 'http://chess-results.com/tnr16264.aspx?lan=1', 'tournament_name': 'CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)', 'dataForSchool': '   10. C.J.D.A.V.SCHOOL(MEERUT) (RtgAvg:1200, TB1: 14 / TB2: 76) Bo.  Name Rtg FED 1 2 3 4 5 6 Pts. Games 1  MANISH KUMAR 0 IND 1 1 1 1 0 1 5,0 6 2  ANSHUL BATHLA 0 IND 1 1 1 1 0 1 5,0 6 3  ARJUN MEHTA 0 IND 1 1 0 0 0 0 2,0 6 4  JALAJ GUPTA 0 IND 1 1 0 0 0 0 2,0 6', 'schoolURLlink': 'http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=2'}


 school name is  L.P. SINGHANIA (KOLKATA)
url for this school is  


 school name is  PENTECOSTAL A.SCHOOL (B.S.CITY)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=25
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
{'_id': '62bc3399418f4d5ed4a1719dfca1317e', 'schoolName': 'PENTECOSTAL A.SCHOOL (B.S.CITY)', 'schoolURL': 'http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=25', 'tournamentURL': 'http://chess-results.com/tnr16264.aspx?lan=1', 'tournament_name': 'CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)', 'dataForSchool': '   19. PENTECOSTAL A.SCHOOL (B.S.CITY) (RtgAvg:1200, TB1: 11,5 / TB2: 72) Bo.  Name Rtg FED 1 2 3 4 5 6 Pts. Games 1  ABHINAV SINHA 0 IND 1 ½ ½ 1 1 1 5,0 6 2  PRAVEEN KUMAR 0 IND 1 1 ½ 1 0 1 4,5 6 3  ABHISHEK ANAND 0 IND 0 0 0 0 0 1 1,0 6 5  ABHISHEK KUMAR 0 IND 0 0 0 1 0 0 1,0 6', 'schoolURLlink': 'http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=25'}


 school name is  AVM, CHAS
url fo


 school name is  R.K.PUBLIC SCHOOL(GARHWA)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=24
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
{'_id': '9aa0a5a2ab609980bdd1bd0a6cae1467', 'schoolName': 'R.K.PUBLIC SCHOOL(GARHWA)', 'schoolURL': 'http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=24', 'tournamentURL': 'http://chess-results.com/tnr16264.aspx?lan=1', 'tournament_name': 'CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)', 'dataForSchool': '   28. R.K.PUBLIC SCHOOL(GARHWA) (RtgAvg:1200, TB1: 9,5 / TB2: 48,5) Bo.  Name Rtg FED 1 2 3 4 5 6 Pts. Games 1  GUNJAN SENGUPTA 0 IND  1 0  0 ½ 1,5 4 2  MADHWENDRA 0 IND - 1 1 - 0 1 3,0 6 3  SUYASH PANDEY 0 IND - 1  - 0 1 2,0 5 4  ABHIJEET PANDEY 0 IND - 1 0 - 0 + 2,0 6 5  AKASH PANDEY 0 IND -  1 -   1,0 3', 'schoolURLlink': 'http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=24'}


 school name is  


 school name is  D.P.S. (DHANBAD)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=1
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
Failure for this tournament   http://chess-results.com/tnr16207.aspx?lan=1

 school name is  SUNBEAM SCHOOL, LAHARTARA (VARANASI)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=3
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
Failure for this tournament   http://chess-results.com/tnr16207.aspx?lan=1

 school name is  BHAGWAT VIDYAPITH (CHAPRA)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=8
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
Failure for this tournament   http://che


 school name is  P.T.J.M.S. VIDYAMANDIR(RAJGIR)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=26
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
Failure for this tournament   http://chess-results.com/tnr16207.aspx?lan=1

 school name is  SOUTH POINT SCHOOL (ASSAM)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=27
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
Failure for this tournament   http://chess-results.com/tnr16207.aspx?lan=1

 school name is  G & HIGH SCHOOL(RANCHI)
url for this school is  http://chess-results.com/tnr16264.aspx?lan=1&art=20&snr=19
getting tournament name
tournament name : CBSE EAST ZONE CHESS CHAMPIONSHIP 2008(UNDER 19 BOYS)
Inserting into Mongodb collection with success
Failure for this tournament   http:/